<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/text_analysis_coref60k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm
!python -m spacy download ru_core_news_md

2021-05-11 22:06:35.484287: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 17.9 MB 1.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
2021-05-11 22:06:45.142138: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 43.9 MB 18 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [43]:
pip install pymorphy2[fast]

In [44]:
pip install Gensim

In [45]:
import spacy
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
nlp = spacy.load('ru_core_news_md')

In [47]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt

--2021-05-11 22:07:02--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt.2’

neuroleptic_antidep 100%[===================>]  11.12M  22.3MB/s    in 0.5s    

2021-05-11 22:07:03 (22.3 MB/s) - ‘neuroleptic_antidepr2.txt.2’ saved [11655409/11655409]



In [48]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt

--2021-05-11 22:07:03--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19072917 (18M) [text/plain]
Saving to: ‘rakpobedim_ru2.txt.2’

rakpobedim_ru2.txt. 100%[===================>]  18.19M  28.8MB/s    in 0.6s    

2021-05-11 22:07:04 (28.8 MB/s) - ‘rakpobedim_ru2.txt.2’ saved [19072917/19072917]



In [49]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-11 22:07:04--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18273113 (17M) [text/plain]
Saving to: ‘hv-info_gepatit2.txt.2’

hv-info_gepatit2.tx 100%[===================>]  17.43M  28.1MB/s    in 0.6s    

2021-05-11 22:07:05 (28.1 MB/s) - ‘hv-info_gepatit2.txt.2’ saved [18273113/18273113]



In [50]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-05-11 22:07:05--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.3’

news_upos_cbow_300_ 100%[===================>] 133.81M  25.6MB/s    in 5.9s    

2021-05-11 22:07:12 (22.5 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.3’ saved [140310878/140310878]



In [51]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()


In [52]:
handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [53]:
handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [54]:
len(my_txt)

27946773

In [55]:
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1,3))

In [56]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-05-11 22:07:13--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.4’

news_upos_cbow_300_ 100%[===================>] 133.81M  25.7MB/s    in 6.1s    

2021-05-11 22:07:19 (22.1 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.4’ saved [140310878/140310878]



In [57]:
posts = my_txt.split('----')

In [58]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]

vectorized_txt = tfidf.fit_transform(sentences_tokenized)
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [59]:
from gensim.models import FastText
from google.colab import files

In [60]:
ft = FastText(sentences_tokenized, size=100, window=5, min_count=5, workers=4,sg=1)

In [61]:
ft.most_similar('алимемазин')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('левомепромазин', 0.9043893218040466),
 ('триоксазин', 0.8729521632194519),
 ('ипрониазид', 0.860264778137207),
 ('пипофезин', 0.8524474501609802),
 ('хлорпромазин', 0.8436838388442993),
 ('зимелидин', 0.8365283012390137),
 ('изокарбоксазид', 0.8309007883071899),
 ('трифлуоперазин', 0.8277693390846252),
 ('палиперидон', 0.8239295482635498),
 ('фуразолидон', 0.8222434520721436)]

In [62]:
ft.wv.__getitem__('алимемазин')

array([-0.27274168,  0.4181226 ,  0.18285716, -0.22400796,  0.12477681,
       -0.03593518,  0.22670269, -0.46027407,  0.12351281, -0.17644699,
       -0.70297384, -0.06741843, -0.15219012,  0.00983881, -0.55092144,
        0.0628059 ,  0.22337158, -0.08198474, -0.21714637, -0.0835813 ,
        0.1142694 , -0.07891117, -0.6026692 ,  0.04257747, -0.0748448 ,
       -0.14176784, -0.24623926, -0.19553408, -0.2906644 , -0.13811412,
        0.02635643,  0.05438923,  0.08266222,  0.16299294, -0.05496788,
       -0.1123174 ,  0.22231229, -0.03640949,  0.03414943,  0.6350578 ,
        0.05441455, -0.07141346, -0.22802822, -0.21617308, -0.56354606,
       -0.22738768,  0.21372214, -0.08945486, -0.5850356 ,  0.12098432,
       -0.21343511,  0.264359  ,  0.05597509, -0.3730376 ,  0.05778533,
        0.16381997, -0.43984488, -0.02010353,  0.5574385 ,  0.08924171,
       -0.30308428, -0.08948301,  0.38816193, -0.29260737, -0.4130248 ,
       -0.01519655,  0.51532537,  0.33951363,  0.15980901,  0.30

In [63]:
# files.download('small_fasttext60k')

In [64]:
# w2v = gensim.models.KeyedVectors.load_word2vec_format('news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [65]:
# model = Word2Vec(size=300, min_count=1)
# model.build_vocab(sentences_tokenized)
# total_examples = model.corpus_count
# model.build_vocab([list(w2v.vocab.keys())], update=True)
# model.intersect_word2vec_format("news_upos_cbow_300_2_2017.bin.gz", binary=True)
# model.train(sentences_tokenized, total_examples=total_examples, epochs=model.iter)

In [66]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('алимемазин')), \
                  np.atleast_2d(ft.wv.get_vector('азалептин')))[0][0]

0.7232068

In [67]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('алимемазин')), \
                  np.atleast_2d(ft.wv.get_vector('стул')))[0][0]

0.33776304

In [68]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('дак')), \
                  np.atleast_2d(ft.wv.get_vector('соф')))[0][0]

0.9028517

In [69]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('дак')), \
                  np.atleast_2d(ft.wv.get_vector('софосбувир')))[0][0]

0.49515676

In [70]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('соф')), \
                  np.atleast_2d(ft.wv.get_vector('софосбувир')))[0][0]

0.50176543

In [71]:
# Модель есть, теперь надо набрать корпус из трех форумов примерно поровну, и чтобы не помереть при разметке.

СОВСЕМ НЕМНОЖКО, для начала!!


In [72]:
short_txt = " "
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
short_txt = handler.read()[:60000]
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:60000]
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:60000]
handler.close()

In [73]:
len(short_txt)

180000

In [74]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [75]:
short_posts = str_list = short_txt.split('----')
short_posts[10]

'\n\nTOPIC\nОбсуждаем препарат Зелдокс (зипрасидон)\n'

In [76]:
len(short_posts)


462

In [77]:
# Словарь предложение:номер поста
tokenizer = RegexpTokenizer(r'\w+')
sent_dict = {}
for num, post in enumerate(short_posts):
  for sent in nltk.sent_tokenize(post):
    sent = ' '.join(tokenizer.tokenize(sent))
    sent = sent.replace('_', '') 
    # sent = nltk.regexp_tokenize(sent, '[а-яА-яa-zA-Z.!? ]')
    if sent:
      sent_dict[sent] = num 

In [78]:
len(sent_dict)

1658

In [79]:
# только для удобства, если не будет влезать в память - использую ключи словаря
sent_list = list(sent_dict.keys())
sent_list[:9]

['TOPIC Обсуждаем препарат Рисперидон Рисполепт Сперидан',
 'НЕ знаю стоило ли мне писать',
 'У меня практически нет никаких проблем и в частности с лекарством',
 'По крайней мере врач говорит что всё идёт как надо',
 'У меня с детства бывают состояния когда я без видимых причин впадаю в панику не управляю своими действиями',
 'Странное беспокойство может продолжаться от нескольких мгновений до нескольких суток',
 'Как то удавалось жить и с этим мириться',
 'Каждый раз придя в себя вынуждена придумывать для окружающих более или менее правдоподобное оправдание своим поступкам',
 'Чувствую себя сторонним наблюдателем собственной жизни']

In [80]:
sent_dict[sent_list[0]]

0

In [81]:

# sentences_tokenized = [w.lower() for w in short_posts]
# sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]
# sentences_tokenized[0]

In [82]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [83]:
sample = 'он'
print('Одуш', morph.parse(sample)[0].tag.animacy)
print('Падеж', morph.parse(sample)[0].tag.case )
print('Пол', morph.parse(sample)[0].tag.gender)
print('Число', morph.parse(sample)[0].tag.number)
print('Лицо', morph.parse(sample)[0].tag.person)
print('Часть речи', morph.parse(sample)[0].tag.POS)
print('Время', morph.parse(sample)[0].tag.tense)
print('Залог', morph.parse(sample)[0].tag.voice)



Одуш None
Падеж nomn
Пол masc
Число sing
Лицо 3per
Часть речи NPRO
Время None
Залог None


In [84]:
aa = morph.parse(sample)[0]
print(aa.tag)

NPRO,masc,3per,Anph sing,nomn


In [85]:
sent_list[61:89]

['Как он переносится',
 'И какие основные побочные',
 'Заранее благодарю',
 'Здравствуйте',
 'Перепробовала АД усиление симптоматики эглонил хороший эффект но началась лактация флюанксол ужасная сонливость теперь первый день пью Зелдокс',
 'Тоже ужасная сонливость прям бессилие просто',
 'Это пройдет',
 'Предварительный диагноз сильный невроз или шизотипическое расстройство астения плохой ночной сон с кошмарами обеднение эмоций и отсутствие энергии тевожность',
 'Помогите подскажите',
 'Устала пить один препарат за другим',
 'Сонливость это нормально в первые дни',
 'Или что делать',
 'Зафира сказал а 9 11 2007 0 02 Сонливость это нормально в первые дни',
 'Я уже 2 недели зелдокс принимаю первую по 40 мг почти не даёт сонливости только небольшое ощущение дереала на пару часов если с утра принять вторую по 80 мг уже на ночь т к',
 'начинаю засыпать часа через 2 3',
 'Планирую повысить до 120 мг',
 'Итог рекомендация производителя о двукратнотном приёме препарата для меня просто невыполн

In [86]:
sent_list[66]

'Тоже ужасная сонливость прям бессилие просто'

In [87]:
doc = nlp(sent_list[66])
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_, 
            [child for child in token.children], morph.parse(token.text)[0].tag,
          [child for child in token.head.children])


Тоже advmod сонливость NOUN [] PRCL [Тоже, ужасная, прям]
ужасная amod сонливость NOUN [] ADJF,Qual femn,sing,nomn [Тоже, ужасная, прям]
сонливость ROOT сонливость NOUN [Тоже, ужасная, прям] NOUN,inan,femn sing,nomn [Тоже, ужасная, прям]
прям nmod сонливость NOUN [бессилие] PRCL [Тоже, ужасная, прям]
бессилие nmod прям ADP [] NOUN,inan,neut sing,nomn [бессилие]
просто ROOT просто PART [] PRCL []


In [88]:
doc = nlp(sent_list[61])
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children], morph.parse(token.text)[0].tag,
          [child for child in token.head.children],)


Как advmod переносится VERB [] CONJ [Как, он]
он nsubj переносится VERB [] NPRO,masc,3per,Anph sing,nomn [Как, он]
переносится ROOT переносится VERB [Как, он] VERB,impf,intr sing,3per,pres,indc [Как, он]


In [89]:
sample = 'он'
print('Одуш', morph.parse(sample)[0].tag.animacy)
print('Падеж', morph.parse(sample)[0].tag.case )
print('Пол', morph.parse(sample)[0].tag.gender)
print('Число', morph.parse(sample)[0].tag.number)
print('Лицо', morph.parse(sample)[0].tag.person)
print('Часть речи', morph.parse(sample)[0].tag.POS)
print('Время', morph.parse(sample)[0].tag.tense)
print('Залог', morph.parse(sample)[0].tag.voice)


Одуш None
Падеж nomn
Пол masc
Число sing
Лицо 3per
Часть речи NPRO
Время None
Залог None


***Разметка для кореференции***

In [90]:
coref_df = pd.DataFrame(columns=['TOKEN', 'TOKEN_VECT', 'TOPIC_NUM', 'POST_NUM', 'ANIMACY', 'CASE', 'GENDER', 'PERSON', 'POS', 'DEPENDENCY', 'HEAD', 'HEAD_VECT' 'HEAD_POS',
                                 'HEAD_ANIMACY', 'HEAD_CASE', 'HEAD_GENDER',  'HEAD_POS', 'SENT_VECTOR', 'HEAD_CHILDS_VECT', 'NER',
                                 'COREFERENCE_CLUSTER', ])

In [91]:
topic_list = short_txt.split('TOPIC')

ImportError: ignored

In [95]:
for topic in topic_list:
  sent_list = topic.split('----')
  for sent in sent_list:
    sent_vector = 0
    post_num = sent_dict.get(sent)
    # sent_vector = sum([model.wv.__getitem__(tok.lower())[-1] for tok in sent.split()])
    
    for i in sent.split():
      try:     
        sent_vector += ft.wv.__getitem__(i.lower())
      except KeyError:
        print(i.lower())
    try:
      sent_vector = sent_vector/len(sent.split())
    except (KeyError, ZeroDivisionError) as e:
        print(e)
    # if sent_vector == '-' or None or 0:
    #   print(sent)
    my_sent = nlp(sent)
    for token in my_sent:
      morphy = morph.parse(token.text)[0]

      app_dict = {}
      app_dict['TOKEN'] = token.text or '-'
      try:
        app_dict['TOKEN_VECT'] = ft.wv.__getitem__(token.text.lower()) if token.text else '-'
      except KeyError:
        print(token.text.lower())
      app_dict['POST_NUM'] = post_num or '-'
      app_dict['ANIMACY'] = morphy.tag.animacy or '-'
      app_dict['CASE'] = morphy.tag.case or '-'
      app_dict['GENDER'] = morphy.tag.gender or '-'
      app_dict['PERSON'] = morphy.tag.person or '-'
      app_dict['POS'] = morphy.tag.POS or '-' 
      app_dict['DEPENDENCY'] = token.dep_ or '-'

      app_dict['HEAD'] = token.head.text or '-'
      try:
        app_dict['HEAD_VECT'] = ft.wv.__getitem__(token.head.text.lower()) if token.head.text else '-'
      except KeyError:
        print(token.head.text.lower())
      app_dict['HEAD_POS'] = token.head.pos_ or '-'
      morphy = morph.parse(token.head.text)[0] or '-'
      app_dict['HEAD_ANIMACY'] = morphy.tag.animacy or '-'
      app_dict['HEAD_CASE'] = morphy.tag.case or '-'
      app_dict['HEAD_GENDER'] = morphy.tag.gender or '-'
      # app_dict['HEAD_PERSON'] = morphy.tag.person or '-'
      app_dict['HEAD_POS'] = morphy.tag.POS or '-'
      app_dict['SENT_VECTOR'] = sent_vector
      try:
        if token.head.children:
          app_dict['HEAD_CHILDS_VECT'] = 0
          for child in token.head.children:
            app_dict['HEAD_CHILDS_VECT'] += ft[child.text.lower()]
          if list(token.head.children):
            app_dict['HEAD_CHILDS_VECT'] = app_dict['HEAD_CHILDS_VECT']/len(list(token.head.children))
      except (KeyError, ZeroDivisionError):
        print(child.text.lower())

      app_dict['NER'] = 0
      app_dict['COREFERENCE_CLUSTER'] = 0
      coref_df = coref_df.append(app_dict, ignore_index=True)


division by zero












(
(
,
,
(
)
(




(а
-
-


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).






.
.
,
,
.
,
,
,
.
.
.
.
.
.
,
,
,
,
,
,
,
,
.
.
.
.
.
.
,
,
,
,
.
,
.
.
.
.
.
.
,
,
,
,
,
.
,
,
,
,
,
,
.
.
.
.
.
.
.
-
-
-
-
-
-
.
-
.
,
,
.
,
,
,
.
.
.
.
.
.
.
.
.
.
.
.
.
(
(
(
(
(
)
(
.
.
.
.
.
.
.
.




.
.
.
.
.
.
.
.
.
.
,
,
.
,
,
.
.
.
.
.
"
"
"
"
"
.
.
(
(
"
(
"
(
"
"
...
(
)
(
.
.
.
.
.
.
.
,
,
.
.
,
,
,
.
,
,
,
,
,
,
,
,
.
.
.
.
.
:
:
:
:
,
,
,
:
,
,
,
,
:
.
.
.
.
,
,
,
,
,
,
,
,
,
,
(
(
)
)
.
)
)
.
.
-
-
-
-
-
!
-


-
.
.
.
.
.
.
.
.
.
.
.
.


.
?
?
?
?
?
,
,
,
?
,
(
(
(
(
)
(
?
?









.
.
.
.




.
.
.
,
,
,
.
,
.
.


.
!
!
!
!
!
!
!
!
!
!
-
-
-
-
-
,
,
,
,
,
,
-
,
,
,
,
,
,
,
,
,
,
,
,
.
,


,
.
,
,
.
.
.
,
,
,
.
,
.
.
.
.
.
.
,
,
,
,
.
.
.


.
и,
-
p.s.








...
...
)
...




?
?
?
?
?
,
,
,
?
,
(
(
(
(
)
(
?
?


?
.
,
,
,
.
-
-
-
-
.
.
,
,
,
,
,
,
,
,
.
,
.
.
.
.
.
.
.
.
.
,
,
,
,
,
,
-
,
.
.
.
.
.
,
.
,
,
,
.
,
,
,
,
,
,
,
.
.
.
.
.
.
.
,
,
.
,
,
,
,
.
.




p.s.




,
,


,
,
,
,
,
,
?


?
-
-
-
-
?
?
?


?




!
!
!
!
?
?
!
!
.
.
.
.
,
,
,
.
.




.
,
,
.


KeyboardInterrupt: ignored

In [ ]:
coref_df_anns = coref_df.loc[:, ['TOKEN', 'TOPIC_NUM', 'POST_NUM', 'HEAD', 'NER', 'COREFERENCE_CLUSTER' ]]

In [ ]:
coref_df.to_csv('coref_fasttext60k.csv', sep=' ', index=False)

In [ ]:
coref_df_anns.to_csv('coref_df_anns60k.csv', sep=' ', index=False)

In [ ]:
from google.colab import files

In [ ]:
files.download('coref_fasttext60k.csv')

In [ ]:
files.download('coref_df_anns60k.csv')